# Google Search Grounding with the Gemini API

This notebook introduces **search grounding** as a method for connecting language model outputs to **verifiable external sources**. Instead of relying solely on the model’s internal knowledge, grounding allows the model to retrieve and reference up-to-date information from Google Search, making it possible to inspect and verify the evidence used in an answer.

Conceptually, search grounding is similar to Retrieval-Augmented Generation (RAG), but the retrieval step is handled automatically by the Gemini API. The model generates search queries, retrieves relevant documents from the web, and returns responses together with links to supporting sources. This is particularly useful in research settings where transparency, traceability, and source verification are required.

### Model and access notes

Gemini 2.0 Flash includes access to Google Search grounding under the free tier, making it suitable for experimentation and teaching. Earlier Gemini models require billing to be enabled for grounding features. Regardless of model choice, grounded outputs should be treated as **starting points for verification**, not as authoritative evidence.

This notebook demonstrates how search grounding can support research workflows such as literature scanning, policy analysis, and claim verification, while maintaining an auditable link between generated text and underlying sources.


## Optional: Use Google AI Studio

If you wish to try out grounding with Google Search, follow this section to try it out using the AI Studio interface. Or skip ahead to the `API` section to try the feature here in your notebook.

### Open AI Studio

Start by going to [AI Studio](https://aistudio.google.com/prompts/new_chat). You should be in the "New chat" interface.

Search Grounding is best with `gemini-2.0-flash`, but try out `gemini-1.5-flash` too.

![New chat in AI Studio](https://storage.googleapis.com/generativeai-downloads/kaggle/ais-newchat.png)

### Ask a question

Now enter a prompt into the chat interface. Try asking something that is timely and might require recent information to answer, like a recent sport score. For this query, grounding will be **disabled** by default.


![Sample question-answer pair without grounding](https://storage.googleapis.com/generativeai-downloads/kaggle/cricket-ungrounded.png)

### Enable grounding

On the right-hand sidebar, under the `Tools` section. Find and enable the `Grounding` option.

![Enable grounding button](https://storage.googleapis.com/generativeai-downloads/kaggle/enable-grounding.png)

Now re-run your question by hovering over the user prompt in the chat history, and pressing the Gemini ✨ icon to re-run your prompt.

![Re-run prompt button](https://storage.googleapis.com/generativeai-downloads/kaggle/re-run-button.png)

You should now see a response generated that references sources from Google Search.

![Response with grounded sources from Google!](https://storage.googleapis.com/generativeai-downloads/kaggle/cricket-grounded.png)


### Try your own queries

Explore this interface and try some other queries. Share what works well in the [Discord](https://discord.com/channels/1101210829807956100/1303438361117069363)! You can start from [this blank template](https://aistudio.google.com/app/prompts/1FZtxKLFZIJ1p_0rICu8K2CNIF1tkAnf4) that has search grounding enabled.

The remaining steps require an API key with billing enabled. They are not required to complete this course; if you have tried grounding in AI Studio you are done for this notebook.

## Use the API

Start by installing and importing the Gemini API Python SDK.

In [ ]:
# Uninstall packages from Kaggle base image that are not needed.
!pip uninstall -qy jupyterlab jupyterlab-lsp
# Install the google-genai SDK for this codelab.
!pip install -qU 'google-genai==1.7.0'

In [6]:
from google import genai
from google.genai import types

from IPython.display import Markdown, HTML, display


genai.__version__

'1.61.0'

### Set up your API key

To run the following cell, your API key must be stored it in a [Kaggle secret](https://www.kaggle.com/discussions/product-feedback/114053) named `GOOGLE_API_KEY`.

If you don't already have an API key, you can grab one from [AI Studio](https://aistudio.google.com/app/apikey). You can find [detailed instructions in the docs](https://ai.google.dev/gemini-api/docs/api-key).

To make the key available through Kaggle secrets, choose `Secrets` from the `Add-ons` menu and follow the instructions to add your key or enable it for this notebook.

In [7]:
from google.colab import userdata
from google import genai

API_KEY = userdata.get('GEMINI_API_KEY')
client = genai.Client(api_key=API_KEY)


### Automated retry

In [10]:
# Define a retry policy. The model might make multiple consecutive calls automatically
# for a complex query, this ensures the client retries if it hits quota limits.
from google.api_core import retry

is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

if not hasattr(genai.models.Models.generate_content, '__wrapped__'):
  genai.models.Models.generate_content = retry.Retry(
      predicate=is_retriable)(genai.models.Models.generate_content)

## Use search grounding

### Model support

Search grounding is available in a limited set of models. Find a model that supports it on [the models page](https://ai.google.dev/gemini-api/docs/models/gemini).

In this guide, you'll use `gemini-2.0-flash`.

### Make a request

To enable search grounding, you specify it as a tool: `google_search`. Like other tools, this is supplied as a parameter in `GenerateContentConfig`, and can be passed to `generate_content` calls as well as `chats.create` (for all chat turns) or `chat.send_message` (for specific turns).


In [8]:
# Ask for information without search grounding.
response = client.models.generate_content(
    model="gemini-3-flash-preview",
    contents="When was the most recent update to Australia's national housing affordability policy?"
)

Markdown(response.text)

The most recent and significant update to Australia’s national housing affordability policy occurred on **July 1, 2024**, with the commencement of the new **National Agreement on Social Housing and Homelessness (NASHH)**.

This agreement replaced the previous National Housing and Homelessness Agreement (NHHA) and represents a major structural shift in how the Federal Government funds states and territories for housing.

Here is a breakdown of the most recent updates and policy milestones:

### 1. The National Agreement on Social Housing and Homelessness (July 1, 2024)
This is a **$9.3 billion, five-year agreement** between the Commonwealth and the States and Territories.
*   **Purpose:** To provide long-term funding for social housing and homelessness services.
*   **Key Update:** It includes a significant increase in funding (roughly $400 million extra per year) compared to the previous agreement, specifically targeted at homelessness services and social housing maintenance.

### 2. The "Homes for Australia" Plan (May 2024 Budget)
In the May 2024 Federal Budget, the Albanese Government announced a **$11.3 billion** housing package. Key updates included:
*   **Rent Assistance:** A further 10% increase in the maximum rates of Commonwealth Rent Assistance (following a 15% increase in 2023).
*   **Infrastructure Fund:** The creation of a **$1 billion** fund for states and territories to fast-track the building of infrastructure (sewerage, water, roads) needed to unlock new housing developments.
*   **Social Housing:** An additional $1 billion specifically for crisis accommodation and housing for women and children fleeing domestic violence.

### 3. The Housing Australia Future Fund (HAFF) Operations (2024)
While the legislation for the HAFF passed in **September 2023**, the first round of funding applications closed in **early 2024**.
*   This is a $10 billion investment fund where the returns are used to build 30,000 new social and affordable rental homes over five years.
*   The first round of successful projects and funding allocations are currently being rolled out as of mid-2024.

### 4. Help to Buy Scheme (Ongoing Legislation)
As of late 2024, the government is still attempting to pass the **"Help to Buy" Bill**.
*   This is a "shared equity" scheme where the government would provide up to 40% of the purchase price for new homes.
*   This remains a point of political contention in the Senate and has not yet been fully implemented nationally.

### 5. National Housing and Homelessness Plan (Upcoming - Late 2024)
The government is currently finalizing a **10-year National Housing and Homelessness Plan**. 
*   This is intended to be the "master document" that coordinates policy across all levels of government, industry, and the community sector. 
*   The final version is expected to be released in **late 2024** following extensive public consultation that concluded in late 2023.

### Summary
If you are looking for the "most recent" change, it is the **July 1, 2024** implementation of the **NASHH** agreement and the **Rent Assistance** increases that took effect following the May 2024 Budget.

Now try with grounding enabled.


In [ ]:
# And now re-run the same query with search grounding enabled.
config_with_search = types.GenerateContentConfig(
    tools=[types.Tool(google_search=types.GoogleSearch())],
)

def query_with_grounding():
    response = client.models.generate_content(
        model="gemini-3-flash-preview",
        contents="When was the most recent update to Australia's national housing affordability policy?",
        config=config_with_search,
    )
    return response.candidates[0]


rc = query_with_grounding()
Markdown(rc.content.parts[0].text)

### Response metadata

When search grounding is used, the model returns extra metadata that includes links to search suggestions, supporting documents and information on how the supporting documents were used.

Each "grounding chunk" represents information retrieved from Google Search that was used in the grounded generation request. Following the URI will take you to the source.

In [ ]:
while not rc.grounding_metadata.grounding_supports or not rc.grounding_metadata.grounding_chunks:
    # If incomplete grounding data was returned, retry.
    rc = query_with_grounding()

chunks = rc.grounding_metadata.grounding_chunks
for chunk in chunks:
    print(f'{chunk.web.title}: {chunk.web.uri}')

As part of the response, there is a standalone styled HTML content block that you use to link back to relevant search suggestions related to the generation.

In [ ]:
HTML(rc.grounding_metadata.search_entry_point.rendered_content)

The `grounding_supports` in the metadata provide a way for you to correlate the grounding chunks used to the generated output text.

In [ ]:
from pprint import pprint

supports = rc.grounding_metadata.grounding_supports
for support in supports:
    pprint(support.to_json_dict())

These supports can be used to highlight text in the response, or build tables of footnotes.

In [ ]:
import io

markdown_buffer = io.StringIO()

# Print the text with footnote markers.
markdown_buffer.write("Supported text:\n\n")
for support in supports:
    markdown_buffer.write(" * ")
    markdown_buffer.write(
        rc.content.parts[0].text[support.segment.start_index : support.segment.end_index]
    )

    for i in support.grounding_chunk_indices:
        chunk = chunks[i].web
        markdown_buffer.write(f"<sup>[{i+1}]</sup>")

    markdown_buffer.write("\n\n")


# And print the footnotes.
markdown_buffer.write("Citations:\n\n")
for i, chunk in enumerate(chunks, start=1):
    markdown_buffer.write(f"{i}. [{chunk.web.title}]({chunk.web.uri})\n")


Markdown(markdown_buffer.getvalue())

## Search with tools

In this example, you'll use enable the Google Search grounding tool and the code generation tool across two steps. In the first step, the model will use Google Search to find the requested information and then in the follow-up question, it generates code to plot the results.

This usage includes textual, visual and code parts, so first define a function to help visualise these.

In [ ]:
from IPython.display import display, Image, Markdown

def show_response(response):
    for p in response.candidates[0].content.parts:
        if p.text:
            display(Markdown(p.text))
        elif p.inline_data:
            display(Image(p.inline_data.data))
        else:
            print(p.to_json_dict())

        display(Markdown('----'))

Now start a chat asking for some information. Here you provide the Google Search tool so that the model can look up data from Google's Search index.

In [ ]:
config_with_search = types.GenerateContentConfig(
    tools=[types.Tool(google_search=types.GoogleSearch())],
    temperature=0.0,
)

chat = client.chats.create(model='gemini-2.0-flash')

response = chat.send_message(
    message="What were the medal tallies, by top-10 countries, for the 2024 olympics?",
    config=config_with_search,
)

show_response(response)

Continuing the chat, now ask the model to convert the data into a chart. The `code_execution` tool is able to generate code to draw charts, execute that code and return the image. You can see the executed code in the `executable_code` part of the response.

Combining results from Google Search with tools like live plotting can enable very powerful use cases that require very little code to run.

In [ ]:
config_with_code = types.GenerateContentConfig(
    tools=[types.Tool(code_execution=types.ToolCodeExecution())],
    temperature=0.0,
)

response = chat.send_message(
    message="Now plot this as a seaborn chart. Break out the medals too.",
    config=config_with_code,
)

show_response(response)

## Further reading

When using search grounding, there are some specific requirements that you must follow, including when and how to show search suggestions, and how to use the grounding links.  Be sure to read and follow the details in the [search grounding capability guide](https://ai.google.dev/gemini-api/docs/grounding) and the [search suggestions guide](https://ai.google.dev/gemini-api/docs/grounding/search-suggestions).

Also check out some more compelling examples of using search grounding with the Live API in the [cookbook](https://github.com/google-gemini/cookbook/), like [this example that uses Google Maps to plot Search results on a map](https://github.com/google-gemini/cookbook/blob/main/examples/LiveAPI_plotting_and_mapping.ipynb) in an audio conversation, or [this example](https://github.com/google-gemini/cookbook/blob/main/examples/Search_grounding_for_research_report.ipynb) that builds a comprehensive research report.

*- [Mark McD](https://linktr.ee/markmcd)*